# Compute within-subjects average maps

In [ ]:
import os
import gc
import glob
import h5py
import ntpath
import seaborn as sns
import pandas as pd
import nibabel as nb
import numpy as np
import matplotlib.pyplot as plt
from nilearn import plotting
from nilearn.masking import unmask

In [ ]:
list_movie = ['friends']
list_subject = ['sub-01', 'sub-02', 'sub-03', 'sub-04', 'sub-05', 'sub-06']

In [ ]:
bg_path = 'path to the mask'
main_path = 'path to hdf5 files'
output_folder = 'path to R2 maps in hdf5 files'

In [ ]:
def keys(f):
    return [key for key in f.keys()]

# WITHIN-SUBJECT ANALYSIS

In [ ]:
scores_df = pd.DataFrame()

for movie in list_movie:
    
    for sub in list_subject:

        ###  Load mask for each subject
        bg_img_path = os.path.join(bg_path, sub + '_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz')

        list_r2_files_subject = glob.glob(main_path + sub + '_*even*fwhm-5_r2_scores.hdf5', recursive=True)

        # you can add a loop and ass variables for clusters and states and fwhm 
        path_filename = os.path.join(main_path, sub + '_dataset-friends_tasks-s01even_cluster-20_states-120_batches-1_reps-100_fwhm-5_r2_scores.hdf5')

        f = h5py.File(path_filename, 'r')
        
        filenames_list_training = keys(f['training'])
        filenames_list_validation = keys(f['validation'])

        # mean R2 across runs 
        data_r2_avg_runs = []

        # Training
        for  key in filenames_list_training:
            
            r2_map = f['training'][key] 
            
            r2_map_np = r2_map[()]
     
            non_zeros_idx = np.where(r2_map_np > 0) 
            
            avg = np.mean(r2_map_np[non_zeros_idx])
            
            data_r2_avg_runs.append(avg)

        # Validation
        for  key in filenames_list_validation:
            
            r2_map = f['validation'][key] 
            
            r2_map_np = r2_map[()]
     
            non_zeros_idx = np.where(r2_map_np > 0) 
            
            avg = np.mean(r2_map_np[non_zeros_idx])
            
            data_r2_avg_runs.append(avg)
            
        data = {'Average R-squared score' : data_r2_avg_runs, 
                'Level': np.repeat('Within-subjects', len(data_r2_avg_runs)),
                'Movie': np.repeat(movie, len(data_r2_avg_runs)),
                'Subject': np.repeat(sub, len(data_r2_avg_runs))
                }

        scores_df = pd.concat([scores_df, pd.DataFrame(data=data)]) 

In [ ]:
scores_df

scores_df

In [ ]:
scores_df.shape

In [ ]:
scores_df.shape

In [ ]:
ymin = 0.3
ymax = 0.9
figure, ax = plt.subplots(1, 1,figsize=(10,7))

plt.rcParams.update({'font.size': 13})

#sns.set(style="whitegrid")

ax.set_ylim(ymin, ymax) 
ax.set_ylim(ymin, ymax) 

#figure.subplots_adjust(hspace=0.2, wspace=0.2)

In [ ]:
tab10 = "tab10"

m = plt.cm.get_cmap(tab10)


sns.swarmplot(x='Level', y='Average R-squared score', 
                   dodge=True, data=scores_df, alpha=0.5, ax=ax)

sns.boxplot(x='Level', y='Average R-squared score', data=scores_df, 
                dodge=True, showfliers = False, boxprops=dict(alpha=0.3),
                 showmeans=True, ax=ax)


In [ ]:
figure

In [ ]:
filename_figure = os.path.join(output_folder, 'figs','average-r-2-squared-scores-all-movies.png')
figure.savefig(filename_figure)
plt.show()

# Visualize average maps

## Within subject

In [ ]:
 nii_output_filename = os.path.join(output_folder, 'between', sub_ts + '-average-map-r2-'+movie+'-embedding-'+sub_eb+'.nii')
            nb.save(avg_map, nii_output_filename)

In [ ]:
plotting.view_img(avg_map,vmax=1,bg_img=bg_img)